In [2]:
import pandas as pd

df_mvp = pd.read_csv("./data/mvps.csv", delimiter=";", index_col=0)
df_team = pd.read_csv("./data/NbaFullTeamsLimpia.csv", delimiter=";", index_col=0)
df_player = pd.read_csv("./data/Nba_players.csv", delimiter=";", index_col=0)

## Limpiamos los DataFrame

In [3]:
# obtenemos el año a partir de la temporada
df_team["Year"] = [int(s[:4]) for s in df_team["Season"]]

# años con información del MVP
years = set(df_mvp["Year"])

# eliminamos la informacion de jugadores y equipos sin informacion del MVP
df_team = df_team[df_team["Year"].isin(years)]
df_player = df_player[df_player["Year"].isin(years)]

# limpiamos nombres de los jugadores
df_player["Player"] = [p.replace('*', '') for p in df_player["Player"]]

# corregimos los nombre de equipo 
dic_teams = {
    'BRK':'NJN' ,
    'CHH': 'CHA',
    'CHO': 'CHA',
    'KCK':'SAC', 
    'NOK':'NOH', 
    'NOP':'NOH',
    'SDC':'WAS',
    'SEA':'OKC',
    'TOT':'TOT',
    'VAN':'MEM',
    'WSB':'WAS',
}
df_player["Team"] = df_player["Tm"].replace(to_replace=dic_teams)
df_mvp["Team"] = df_mvp["Tm"].replace(to_replace=dic_teams)

# cambiamos posición por un número
dic_posiciones = {
    'C': 5,
    'PF': 4, 
    'SF': 3,
    'SG': 2,
    'PG': 1, 
}
df_player["Pos"] = df_player["Pos"].replace(to_replace=dic_posiciones)

In [4]:
def quitaT(s):
    return s.replace('T', '')
df_mvp["Rank"] = df_mvp["Rank"].map(quitaT)

In [5]:
df_player.reset_index(drop=True, inplace=True)
df_team.reset_index(drop=True, inplace=True)
df_mvp.reset_index(drop=True, inplace=True)

## Eliminamos filas de:

* Player: los jugadores de 'CHA' de 2002 porque no hay información global de ese equipo ese año. Afortunadamente ningún jugador de CHA en 2002 fue candidato a MVP.
* Player: los jugadores de 'TOT' porque han jugado en más de un equipo el mismo año.
* MVP: la información de MVP de los jugadores 'TOT', no importa porque tienen un Share muy pequeño.

In [6]:
df_player.drop(df_player[(df_player["Team"] == "CHA") & (df_player["Year"] == 2002)].index, inplace=True)
df_player.drop(df_player[df_player["Team"] == "TOT"].index, inplace=True)
df_mvp.drop(df_mvp[df_mvp["Team"] == "TOT"].index, inplace=True)

In [7]:
df_mvp

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year,Team
0,1,Larry Bird,27,BOS,52.0,652.0,760,0.858,79,38.3,...,6.6,1.8,0.9,0.492,0.247,0.888,13.6,0.215,1984,BOS
1,2,Bernard King,27,NYK,11.0,373.0,760,0.491,77,34.6,...,2.1,1.0,0.2,0.572,0.000,0.779,12.1,0.218,1984,NYK
2,3,Magic Johnson,24,LAL,5.0,305.0,760,0.401,67,38.3,...,13.1,2.2,0.7,0.565,0.207,0.810,10.2,0.191,1984,LAL
3,4,Kareem Abdul-Jabbar,36,LAL,3.0,153.0,760,0.201,80,32.8,...,2.6,0.7,1.8,0.578,0.000,0.723,8.9,0.163,1984,LAL
4,5,Isiah Thomas,22,DET,1.0,115.0,760,0.151,82,36.7,...,11.1,2.5,0.4,0.462,0.338,0.733,9.1,0.144,1984,DET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,8,Stephen Curry,33,GSW,0.0,4.0,1000,0.004,64,34.5,...,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173,2022,GSW
605,9,Chris Paul,36,PHO,0.0,2.0,1000,0.002,65,32.9,...,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210,2022,PHO
606,10,DeMar DeRozan,32,CHI,0.0,1.0,1000,0.001,76,36.1,...,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154,2022,CHI
607,10,Kevin Durant,33,BRK,0.0,1.0,1000,0.001,55,37.2,...,6.4,0.9,0.9,0.518,0.383,0.910,8.4,0.198,2022,NJN


In [8]:
# borrar columnas que no necesitamos
df_player.drop(columns=["Tm"], inplace=True)
df_mvp.drop(columns=[
    "Tm", "G", "FG%", "FT%", "3P%", "TRB", "AST", "STL", "BLK", "Age", "MP", "PTS", "WS", "WS/48"], inplace=True)
df_team.drop(columns=["Season", "Lg"], inplace=True)

In [9]:
# cambiar el nombre a las columnas de team con el mismo nombre que las de player

columnas_comunes = ['2P', '2P%', '2PA', '3P', '3P%', '3PA', 'AST', 'Age', 'BLK', 'DRB', 'FG', 'FG%', 'FGA', 'FT', 'FT%', 'FTA', 'G', 'MP', 'ORB', 'PF', 'PTS', 'STL', 'TOV', 'TRB', 'Ht.', 'Wt.']
cambio = {}
for col in columnas_comunes:
    cambio[col] = "T-" + col
df_team = df_team.rename(columns=cambio)

In [10]:
print("player:", len(df_player), df_player.columns)
print("mvp:", len(df_mvp), df_mvp.columns)
print("team:", len(df_team), df_team.columns)

player: 19078 Index(['Player', 'Pos', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'Team'],
      dtype='object')
mvp: 602 Index(['Rank', 'Player', 'First', 'Pts Won', 'Pts Max', 'Share', 'Year',
       'Team'],
      dtype='object')
team: 1110 Index(['W', 'L', 'Finish', 'T-Age', 'T-Ht.', 'T-Wt.', 'T-G', 'T-MP', 'T-FG',
       'T-FGA', 'T-FG%', 'T-3P', 'T-3PA', 'T-3P%', 'T-2P', 'T-2PA', 'T-2P%',
       'T-FT', 'T-FTA', 'T-FT%', 'T-ORB', 'T-DRB', 'T-TRB', 'T-AST', 'T-STL',
       'T-BLK', 'T-TOV', 'T-PF', 'T-PTS', 'Team', 'Year'],
      dtype='object')


In [11]:
print("columnas comunes 'player' y 'mvp':", sorted(set(df_player).intersection(df_mvp)))
print("columnas comunes 'player' y 'team':", sorted(set(df_player).intersection(df_team)))

columnas comunes 'player' y 'mvp': ['Player', 'Team', 'Year']
columnas comunes 'player' y 'team': ['Team', 'Year']


## Unimos los jugadores con su equipo esa temporada

In [12]:
# No hay resultados globales de CHA en la temporada 2002
df_pt = pd.merge(df_player, df_team, on=["Team", "Year"], how='inner')
len(df_player), len(df_team), len(df_mvp), len(df_pt)

(19078, 1110, 602, 19078)

## A los anteriores unimos la información de los candidatos a mvp
Los jugadores que no son candidatos tendrán toda la información de mvp como NaN

In [13]:
df_final = pd.merge(df_pt, df_mvp, on=["Team", "Year", "Player"], how="left")
len(df_player), len(df_team), len(df_mvp), len(df_pt), len(df_final)

(19078, 1110, 602, 19078, 19078)

In [14]:
print("Jugadores SIN información MVP:", df_final["Share"].isnull().sum())
print("Jugadores CON información MVP:", df_final["Share"].notnull().sum())
print("TOTAL Jugadores:", len(df_final))

Jugadores SIN información MVP: 18476
Jugadores CON información MVP: 602
TOTAL Jugadores: 19078


## Ponemos a 0 todos los NaN

In [15]:
df_final.fillna(0, inplace=True)

In [16]:
df_final

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,...,T-STL,T-BLK,T-TOV,T-PF,T-PTS,Rank,First,Pts Won,Pts Max,Share
0,Kareem Abdul-Jabbar,5,36,80,80.0,2622.0,716,1238,0.578,0.0,...,695.0,481.0,1537.0,1931,9696,4,3.0,153.0,760.0,0.201
1,Michael Cooper,2,27,82,9.0,2387.0,273,549,0.497,38.0,...,695.0,481.0,1537.0,1931,9696,0,0.0,0.0,0.0,0.000
2,Calvin Garrett,3,27,41,0.0,478.0,78,152,0.513,2.0,...,695.0,481.0,1537.0,1931,9696,0,0.0,0.0,0.0,0.000
3,Magic Johnson,1,24,67,66.0,2567.0,441,780,0.565,6.0,...,695.0,481.0,1537.0,1931,9696,3,5.0,305.0,760.0,0.401
4,Eddie Jordan,1,29,3,0.0,27.0,4,8,0.500,0.0,...,695.0,481.0,1537.0,1931,9696,0,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19073,Daniel Theis,5,29,21,6.0,393.0,67,112,0.598,10.0,...,521.0,430.0,1095.0,1542,9671,0,0.0,0.0,0.0,0.000
19074,Brodric Thomas,2,25,12,0.0,60.0,8,18,0.444,2.0,...,521.0,430.0,1095.0,1542,9671,0,0.0,0.0,0.0,0.000
19075,Derrick White,2,27,26,4.0,713.0,94,230,0.409,34.0,...,521.0,430.0,1095.0,1542,9671,0,0.0,0.0,0.0,0.000
19076,Grant Williams,4,23,77,21.0,1875.0,205,432,0.475,106.0,...,521.0,430.0,1095.0,1542,9671,0,0.0,0.0,0.0,0.000


In [17]:
df_final.columns

Index(['Player', 'Pos', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'Team',
       'W', 'L', 'Finish', 'T-Age', 'T-Ht.', 'T-Wt.', 'T-G', 'T-MP', 'T-FG',
       'T-FGA', 'T-FG%', 'T-3P', 'T-3PA', 'T-3P%', 'T-2P', 'T-2PA', 'T-2P%',
       'T-FT', 'T-FTA', 'T-FT%', 'T-ORB', 'T-DRB', 'T-TRB', 'T-AST', 'T-STL',
       'T-BLK', 'T-TOV', 'T-PF', 'T-PTS', 'Rank', 'First', 'Pts Won',
       'Pts Max', 'Share'],
      dtype='object')

## Guardar el Dataframe

In [18]:
df_final.to_csv("./data/predecir_mvp.csv", index=False, sep=";")